https://www.kaggle.com/code/jaeyoungleejoin/pbl-kobert-finetuning-v3

https://www.kaggle.com/code/jaeyoungleejoin/pbl-inference

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files

# Kaggle API 키 업로드
uploaded = files.upload()

# 업로드된 kaggle.json 파일을 /root/.kaggle 디렉토리로 이동
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!pip install torch gradio

In [ ]:
!pip install transformers==4.36.1 accelerate==0.24.0 bitsandbytes==0.41.3.post2 datasets==2.15.0 peft==0.7.0 trl==0.7.4 sentence-transformers==2.2.2 hydra-core pynvml sentencepiece einops

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-vt4lmyta/kobert-tokenizer_6b0d3c358b8749f1bc3e1ef725dac153
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-vt4lmyta/kobert-tokenizer_6b0d3c358b8749f1bc3e1ef725dac153
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
# 필요한 디렉토리 생성
!mkdir -p datasets/external

# 데이터셋과 모델 다운로드 및 압축 해제
!cd datasets && kaggle datasets download -d jaeyoungleejoin/utils-sent-v1 && unzip utils-sent-v1.zip -d ./ && rm utils-sent-v1.zip
!cd datasets && kaggle datasets download -d jaeyoungleejoin/amazon-sentiment-v3 && unzip amazon-sentiment-v3.zip -d ./ && rm amazon-sentiment-v3.zip
!cd datasets && kaggle datasets download -d jaeyoungleejoin/pblsemi && unzip pblsemi.zip -d ./ && rm pblsemi.zip

  0% 0.00/47.3k [00:00<?, ?B/s]
100% 47.3k/47.3k [00:00<00:00, 28.3MB/s]
Archive:  utils-sent-v1.zip
replace ./generate_r_clm.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./generate_r_clm.py     
  inflating: ./generate_r_clm_from_scratch.py  
  inflating: ./r_clm/ai_dataset.py   
  inflating: ./r_clm/ai_loader.py    
  inflating: ./r_clm/ai_optimizer.py  
  inflating: ./r_detect/ai_dataset.py  
  inflating: ./r_detect/ai_loader.py  
  inflating: ./r_detect/ai_model.py  
  inflating: ./r_detect/ai_optimizer.py  
  inflating: ./r_embed/ai_dataset.py  
  inflating: ./r_embed/ai_loader.py  
  inflating: ./r_embed/ai_model.py   
  inflating: ./r_embed/ai_optimizer.py  
  inflating: ./r_kobert/ai_dataset.py  
  inflating: ./r_kobert/ai_loader.py  
  inflating: ./r_kobert/ai_model.py  
  inflating: ./r_kobert/ai_optimizer.py  
  inflating: ./r_ranking/ai_dataset.py  
  inflating: ./r_ranking/ai_loader.py  
  inflating: ./r_ranking/ai_model.py  
  inflating: ./r_ranking/ai_optimiz

In [ ]:
from kobert_tokenizer import KoBERTTokenizer

# KoBERT 토크나이저 로드
tokenizer = KoBERTTokenizer.from_pretrained("skt/kobert-base-v1")

# 예시 문장들
sentences = [
    "아주 좋은 물건이군.",
    "오늘 날씨가 참 좋네요.",
    "토크나이저를 사용해서 문장을 변환하는 과정을 설명합니다."
]

# 토크나이저를 사용하여 문장 토큰화 및 인코딩
encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# 인코딩된 결과 출력
print("토큰 ID:\n", encoded_inputs["input_ids"])
print("어텐션 마스크:\n", encoded_inputs["attention_mask"])

# 토큰 ID를 다시 단어 토큰으로 변환하여 출력
tokens = [tokenizer.convert_ids_to_tokens(ids) for ids in encoded_inputs["input_ids"]]
print("단어 토큰:")
for sentence_tokens in tokens:
    print(sentence_tokens)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


토큰 ID:
 tensor([[   2, 3128, 4209, 2137, 7096, 5512,   54,    3,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1],
        [   2, 3419, 1408, 5330, 4427, 4204, 5703,   54,    3,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1],
        [   2,  517, 7630, 5655, 7096, 7199, 6116, 2613, 7850, 2120, 7187, 2339,
         7946, 7794, 1067, 2773, 7843,   54,    3]])
어텐션 마스크:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
단어 토큰:
['[CLS]', '▁아주', '▁좋은', '▁물건', '이', '군', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['[CLS]', '▁오늘', '▁날씨', '가', '▁참', '▁좋', '네요', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
['[CLS]', '▁', '토크', '나', '이', '저', '를', '▁사용', '해서', '▁문', '장을', '▁

In [ ]:
# prompt: sys.path.insert(0, '/content/datasets')

import sys
sys.path.insert(0, '/content/datasets')


In [ ]:
# import sys
# sys.path.insert(0, '/content/datasets')

# import argparse
# import os
# import gc

# import pandas as pd
# import torch
# from accelerate import Accelerator
# from omegaconf import OmegaConf
# from peft import PeftModel

# from torch.utils.data import DataLoader
# from tqdm.auto import tqdm
# from transformers import AutoModelForSequenceClassification, BitsAndBytesConfig

# from r_ranking.ai_dataset import AiDataset
# from r_ranking.ai_loader import AiCollator, show_batch
# from r_ranking.ai_model import AiModel

# char_to_remove = ['{', '£', '\x97', '¹', 'å', '\\', '\x85', '<', '\x99', \
#                   'é', ']', '+', 'Ö', '\xa0', '>', '|', '\x80', '~', '©', \
#                   '/', '\x93', '$', 'Ó', '²', '^', ';', '`', 'á', '*', '(', \
#                   '¶', '®', '[', '\x94', '\x91', '#', '-', 'ó', ')', '}', '=']

# def preprocess_text(text):
#     text = text.encode("ascii", "ignore").decode('ascii')
#     text = text.strip()
#     text = text.strip("\"")

#     for c in char_to_remove:
#         text = text.replace(c, "")

#     if text[-1]!=".":
#         text = text.split(".")
#         text = ".".join(text[:-1])
#         text += "."
#     return text


# def run_inference(accelerator, model, infer_dl):#, example_ids):
#     model.eval()
#     all_predictions = []

#     progress_bar = tqdm(range(len(infer_dl)), disable=not accelerator.is_local_main_process)

#     for step, batch in enumerate(infer_dl):
#         with torch.no_grad():
#             logits, _ = model(**batch)

#         logits = logits.reshape(-1)
#         predictions = torch.sigmoid(logits)
#         predictions = accelerator.gather_for_metrics(predictions)
#         predictions = predictions.cpu().numpy().tolist()

#         all_predictions.extend(predictions)

#         progress_bar.update(1)
#     progress_bar.close()

#     result_df = pd.DataFrame()
# #    result_df["id"] = example_ids
#     result_df["generated"] = all_predictions

#     return result_df

# def main(cfg, save_dir, model_id):

#     # create accelerator
#     accelerator = Accelerator()

#     # read test data
#     if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#         test_df = pd.read_csv("/kaggle/input/amazon-sentiment-v3/ama_test.csv", sep=',')
#     else:
#         test_df = pd.read_csv("/kaggle/input/amazon-sentiment-v3/ama_test.csv", sep=',')

#     test_df['text'] = test_df['text'].apply(preprocess_text)
#     accelerator.print(f'Test csv shape: {test_df.shape}')
#     test_df['generated'] = 1 # TODO: NEEDED NOW, FIx it

#     with accelerator.main_process_first():
#         dataset_creator = AiDataset(cfg)
#         infer_ds = dataset_creator.get_dataset(test_df)

#     tokenizer = dataset_creator.tokenizer

#     infer_ds = infer_ds.sort("input_length")
#     infer_ds.set_format(
#         type=None,
#         columns=[
# #             'id',                   # id
#             'input_ids',
#             'attention_mask',
#             'generated'
#         ]
#     )

# #     infer_ids = infer_ds["id"]  # .tolist()

#     #--
#     data_collator = AiCollator(
#         tokenizer=tokenizer,
#         pad_to_multiple_of=64
#     )

#     infer_dl = DataLoader(
#         infer_ds,
#         batch_size=cfg.predict_params.per_device_eval_batch_size,
#         shuffle=False,
#         collate_fn=data_collator,
#     )

#     accelerator.print("data preparation done...")
#     accelerator.print("~~"*40)
#     accelerator.wait_for_everyone()


#     #----------
#     for b in infer_dl:
#         break
#     show_batch(b, tokenizer, task='infer', print_fn=accelerator.print)
#     accelerator.print("~~"*40)
#     #----------
#     # model -----------------------------------------------------------------------------#
#     model = AiModel(cfg, accelerator.device)

#     checkpoint_path = cfg.predict_params.checkpoint_path
#     accelerator.print("=="*50)
#     accelerator.print(f"loading model from checkpoint: {checkpoint_path}")

#     ckpt = torch.load(checkpoint_path)
#     model.load_state_dict(ckpt['state_dict'])
#     del ckpt
#     gc.collect()
#     print("loaded!")
#     accelerator.print("### Loaded Model Weights ###")

#     model, infer_dl = accelerator.prepare(model, infer_dl)

#     # run inference ---
#     sub_df = run_inference(accelerator, model, infer_dl) #, infer_ids)

#     accelerator.wait_for_everyone()

#     if accelerator.is_main_process:
#         save_path = os.path.join(save_dir, f"{model_id}.parquet")
#         sub_df.to_parquet(save_path)
#         accelerator.print("done!")
#         accelerator.print("~~"*40)

# if __name__ == "__main__":
#     ap = argparse.ArgumentParser()
#     ap.add_argument('--config_path', type=str, required=True)
#     ap.add_argument('--save_dir', type=str, required=True)
#     ap.add_argument('--model_id', type=str, required=True)

#     args = ap.parse_args()
#     cfg = OmegaConf.load(args.config_path)

#     os.makedirs(args.save_dir, exist_ok=True)

#     # execution
#     main(
#         cfg,
#         save_dir=args.save_dir,
#         model_id=args.model_id,
#     )

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel, BertTokenizer
import gradio as gr

from transformers import AutoModel
import torch.nn as nn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint
from transformers import AutoConfig, AutoModel
import sys

# @sys.path.insert(0, '/kaggle/input/omegaconf-230/omegaconf-2.3.0')
# @sys.path.insert(0, '/kaggle/input/utils-sent-v1')
from omegaconf import OmegaConf


# #YAML 파일 로드
# cfg = OmegaConf.load('./configs/conf_kobert.yaml')


class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()

    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

# class KoBERTForSequenceClassification(nn.Module):
class AiModel(nn.Module):
    """
    The PBL semi project sentiment detext Model
    """
    def __init__(self):         # num_labels   #cfg, device
        super(AiModel, self).__init__()        # KoBERTForSequenceClassification
        # self.cfg=cfg
        self.num_labels = 1      # 2
        self.backbone = AutoModel.from_pretrained("skt/kobert-base-v1")     # kobert    backbone
        self.dropout = nn.Dropout(0.05)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, 1)   # num_labels : 2
        self.pool = MeanPooling()
#         self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 디바이스 설정
#         self.to(self.device)  # 모델을 해당 디바이스로 이동
        self.loss_fn = nn.BCEWithLogitsLoss()
#             # classifier
#         num_features = self.backbone.config.hidden_size
#         self.classifier = nn.Linear(num_features, 1)

    def encode(
        self,
        input_ids,
        attention_mask,
    ):
        outputs = self.backbone(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=False,
        )

        encoder_layer = outputs.last_hidden_state
        embeddings = self.pool(encoder_layer, attention_mask)

        return embeddings



    def forward(self, input_ids, attention_mask, labels=None,  **kwargs):
        # features
        features = self.encode(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
        features = self.dropout(features)
        logits = self.classifier(features).reshape(-1)

        # loss
        loss = None
        # labels = labels.reshape(-1)
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return logits, loss

#         def forward(self, input_ids, attention_mask):
#         outputs = self.kobert(input_ids=input_ids, attention_mask=attention_mask)
#         last_hidden_state = outputs[0]
#         pooled_output = self.mean_pooling(last_hidden_state, attention_mask)
#         pooled_output = self.dropout(pooled_output)
#         logits = self.classifier(pooled_output)
#         return logits

# 모델 인스턴스화 및 로드
model = AiModel()
checkpoint = torch.load('/content/datasets/detect_ai_model_best.pth (1).tar', map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['state_dict'])
model.eval()


AiModel(
  (backbone): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [ ]:

# tokenizer = BertTokenizer.from_pretrained('skt/kobert-base-v1')
# Gradio 인터페이스 함수 (수정)
def predict(text):
    # model = AiModel()
    # checkpoint = torch.load('/content/drive/MyDrive/PBL중간과제/detect_ai_model_best.pth (1).tar', map_location=torch.device('cpu'))
    # model.load_state_dict(checkpoint['state_dict'])
    # model.eval()
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        logits, _ = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])  # 변경: logits만 사용
    logits = logits.reshape(-1)
    predictions = torch.sigmoid(logits)
    # predictions = accelerator.gather_for_metrics(predictions)
    # predictions = predictions.cpu().numpy().tolist()
    prediction = predictions.item() > 0.5  # 이진 분류 임계값 적용
    return "positive" if prediction else "negative"

# Gradio 앱 생성 (API 변경사항 적용)
iface = gr.Interface(fn=predict,
                     inputs=gr.Textbox(lines=2, placeholder="Text to classify..."),
                     outputs=gr.Text(),
                     title="KoBERT Text Classification",
                     description="Enter a Korean text to classify.")

# 앱 실행
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3f0059b315c2c2b1e4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# 테스트 케이스 예시
test_cases = [
    ("오늘 아주 좋은 물건을 샀어","positive"),
    ("이건 아주 별로였어. 다신 안 살거야","negative"),
    # 여기에 더 많은 테스트 케이스를 추가할 수 있습니다.
]

# predict 함수를 사용하여 각 테스트 케이스 평가
for text, expected in test_cases:
    prediction = predict(text)
    print(f"Text: {text}\nPredicted: {prediction}, Expected: {expected}\n")

Text: 오늘 아주 좋은 물건을 샀어
Predicted: positive, Expected: positive

Text: 이건 아주 별로였어. 다신 안 살거야
Predicted: negative, Expected: negative

